# Video Preprocessing

Description

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import json

In [11]:
from video_analyzer import VideoAnalyzer, get_processing_params
from preprocessor import Preprocessor

In [12]:
timestamp = "04142025"
motion_version = "versionB"
pose_version = "versionB"
preprocessing_version = "v3"
path_to_root = "/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/"

In [13]:
metadata = pd.read_csv(os.path.join(
    path_to_root,
    "data",
    "raw",
    "combined",
    "target_dataset_video_metadata.csv"
    ))

In [14]:
for i, metadata_row in metadata.iterrows():
    print(f"\rProcessing video {i+1} of {len(metadata)}: {metadata_row.filename}", end="")
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
    analyzer = VideoAnalyzer(
        metadata_row,
        timestamp,
        path_to_root,
        verbose=False,
        motion_detection_version=motion_version,
        pose_detection_version=pose_version
    )
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

    motion_data = analyzer.motion_detect()
    motion_result = analyzer.motion_analyze()

    pose_data = analyzer.pose_detect()
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    pose_result = analyzer.pose_analyze()
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

    analyzer.save_analysis_info()

Processing video 1 of 150: ajudar_ne_1.mp4window_duration: 0.2 seconds, at 12.0 fps = 2 frame window_size
Loaded pose estimation results from versionB


W0000 00:00:1744637525.055585   44537 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.094633   44537 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.096893   44540 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.098449   44537 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.104383   44536 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.129988   44540 inference_feedback_manager.cc:114] Feedback manager 

Processing video 2 of 150: ajudar_sb_2.mp4window_duration: 0.2 seconds, at 29.97002997002997 fps = 5 frame window_size
Loaded pose estimation results from versionB


W0000 00:00:1744637525.268098   44545 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.364986   44544 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.367002   44545 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.367369   44547 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.367866   44544 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637525.382098   44545 inference_feedback_manager.cc:114] Feedback manager 

Processing video 3 of 150: ajudar_uf_3.mp4window_duration: 0.2 seconds, at 29.97002997002997 fps = 5 frame window_size
Loaded pose estimation results from versionB
Processing video 4 of 150: ajudar_vl_4.mp4window_duration: 0.2 seconds, at 29.97002997002997 fps = 5 frame window_size
Loaded pose estimation results from versionB


...

W0000 00:00:1744637544.682992   45760 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.704061   45760 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.706107   45761 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.706422   45758 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.707039   45760 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.720191   45761 inference_feedback_manager.cc:114] Feedback manager 

Processing video 147 of 150: vagina_uf_3.mp4window_duration: 0.2 seconds, at 29.97002997002997 fps = 5 frame window_size
Loaded pose estimation results from versionB
Processing video 148 of 150: vagina_vl_4.mp4window_duration: 0.2 seconds, at 29.97002997002997 fps = 5 frame window_size
Loaded pose estimation results from versionB


W0000 00:00:1744637544.943205   45775 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.964862   45775 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.966999   45773 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.967486   45774 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.967720   45775 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637544.980434   45773 inference_feedback_manager.cc:114] Feedback manager 

Processing video 149 of 150: vagina_vl_5.mp4window_duration: 0.2 seconds, at 29.97002997002997 fps = 5 frame window_size
Loaded pose estimation results from versionB
Processing video 150 of 150: vagina_vl_6.mp4window_duration: 0.2 seconds, at 60.0 fps = 12 frame window_size
Loaded pose estimation results from versionB


W0000 00:00:1744637545.225745   45789 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637545.249468   45790 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637545.252025   45794 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637545.252025   45793 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637545.252621   45795 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744637545.268535   45794 inference_feedback_manager.cc:114] Feedback manager 

In [7]:
for i, metadata_row in metadata.iterrows():
    print(f"\rProcessing video {i+1} of {len(metadata)}: {metadata_row.filename}", end="")
    with open(os.path.join(
        path_to_root, 
        "data", 
        "interim", 
        "Analysis",
        f"{timestamp}_motion{motion_version}_pose{pose_version}", 
        metadata_row["filename"].split(".")[0] + "_analysis_info.json"
        )) as f:
        analysis_info = json.load(f)
    
    preprocessing_params = get_processing_params(analysis_info)

    preprocessor = Preprocessor(
        metadata_row,
        preprocessing_params,
        path_to_root,
        preprocess_version='v2',
        verbose=False,
        save_intermediate=False,
    )

    preprocessor.preprocess_video()
    preprocessor.preprocess_landmarks()


Processing video 156 of 156: vagina_vl_6.mp44